In [ ]:
# Environment setup and repository cloning
import os
import sys
import warnings
import subprocess

# Set matplotlib backend to avoid ipympl conflicts
os.environ['MPLBACKEND'] = 'Agg'

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set JAX to use CPU to avoid GPU memory issues if needed
os.environ['JAX_PLATFORM_NAME'] = 'cpu'

# Clone the OI-QDax repository (leap_hand branch) if not already present
repo_url = "https://github.com/EduardGilM/OI-QDax.git"
repo_name = "OI-QDax"
branch_name = "leap_hand"

if not os.path.exists(repo_name):
    print(f"🔄 Cloning repository {repo_url} (branch: {branch_name})...")
    try:
        subprocess.run(["git", "clone", "-b", branch_name, repo_url], check=True)
        print("✅ Repository cloned successfully!")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error cloning repository: {e}")
        print("📝 Please ensure git is installed and accessible")
        print("🔄 Trying to clone without specifying branch...")
        try:
            subprocess.run(["git", "clone", repo_url], check=True)
            # Switch to the leap_hand branch
            subprocess.run(["git", "checkout", branch_name], cwd=repo_name, check=True)
            print("✅ Repository cloned and switched to leap_hand branch!")
        except subprocess.CalledProcessError as e2:
            print(f"❌ Error with alternative method: {e2}")
else:
    print("✅ Repository already exists!")
    # Make sure we're on the correct branch
    try:
        subprocess.run(["git", "checkout", branch_name], cwd=repo_name, check=True)
        print(f"✅ Switched to {branch_name} branch!")
    except subprocess.CalledProcessError:
        print(f"⚠️ Could not switch to {branch_name} branch, using current branch")

# Add the repository to Python path
repo_path = os.path.abspath(repo_name)
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)
    print(f"✅ Added {repo_path} to Python path")

print("✅ Environment configured successfully!")

In [ ]:
# Import required libraries for left hand visualization
import jax
import jax.numpy as jnp
from IPython.display import HTML

# Set matplotlib backend before importing brax
import matplotlib
matplotlib.use('Agg')

# Import directly from brax for rendering
from brax.v1.io import html

# Import the left hand environment from the cloned repository
try:
    from qdax.environments.left_hand import LeftHand
    print("✅ Successfully imported LeftHand from cloned repository!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("📝 Make sure the repository was cloned correctly and contains the left_hand environment")
    # Try alternative import paths
    try:
        import sys
        sys.path.append('./OI-QDax')
        from qdax.environments.left_hand import LeftHand
        print("✅ Successfully imported LeftHand with alternative path!")
    except ImportError as e2:
        print(f"❌ Alternative import also failed: {e2}")
        raise

print("✅ All libraries imported successfully!")
print("✅ Left hand environment ready to use!")
print(f"✅ Matplotlib backend set to: {matplotlib.get_backend()}")

✅ All libraries imported successfully!
✅ Left hand environment ready to use!


In [ ]:
# Initialize the Left Hand environment
env = LeftHand()

# Create a random key for initialization
key = jax.random.PRNGKey(42)

# Reset the environment to get initial state
reset_fn = jax.jit(env.reset)
state = reset_fn(key)

print("✅ Left Hand environment initialized!")
print(f"State shape: {jax.tree_map(lambda x: x.shape, state)}")
print(f"Environment step size: {env.sys.dt}")

In [ ]:
# Create render of the Left Hand
# Generate a simple trajectory by taking a few steps with random actions
states = [state]
current_state = state

# Take a few steps to see the hand in motion
for i in range(10):
    # Generate random action
    action = jax.random.normal(key, shape=(env.action_size,)) * 0.1
    key, subkey = jax.random.split(key)
    
    # Step the environment
    current_state = env.step(current_state, action)
    states.append(current_state)

print(f"✅ Generated trajectory with {len(states)} states")

# Create HTML visualization
html_string = html.render(env.sys, states)

# Display the render
HTML(html_string)

# Left Hand Visualization

The render above shows the LEAP Hand (Left Hand) in action. The visualization includes:

- **Hand Model**: A detailed 3D model of the left hand with all joints and links
- **Animation**: Shows the hand performing random movements over several time steps
- **Physics**: Realistic physics simulation using the Brax physics engine

You can interact with the 3D visualization by:
- Rotating the view by clicking and dragging
- Zooming in/out with the mouse wheel
- Playing/pausing the animation with the controls

The hand model is based on the LEAP Hand robot design and includes all the necessary degrees of freedom for complex manipulation tasks.